In [ ]:
import numpy as np
import pandas as pd
import spacy
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm import tqdm
from torch.nn import GRU, ModuleList, Linear
from torch.nn.functional import sigmoid, softmax, relu
from torch.optim import Adagrad

In [2]:
nlp = spacy.load('/home/adarsh/DLNLP/spacy/glove_840b_300d')
filepath = '/home/adarsh/DLNLP/datasets/Assignment2/dataset.csv'

In [7]:
class ReviewDataset(Dataset):
    def __init__(self, filepath, nlp) -> None:
        super(ReviewDataset, self).__init__()
        self.nlp = nlp
        dataset = pd.read_csv(filepath)
        docs = [ nlp(review) for review in tqdm(dataset['review'])]
        self.X = [ torch.stack([ torch.tensor(token.vector) for token in doc ]) for doc in tqdm(docs) ]
        self.Y = torch.tensor(np.where(dataset['sentiment']=='negative', 0, 1))
    
    def __getitem__(self, index):
        return self.X[index].cuda(), self.Y[index].cuda()
    
    def __len__(self):
        return len(self.X)

dataset = ReviewDataset(filepath, nlp)
    



















 66%|██████▌   | 26286/40000 [03:19<01:44, 131.68it/s]










KeyboardInterrupt: 

In [4]:
class RecurrentClassifier(torch.nn.Module):

    def __init__(self) -> None:
        super(RecurrentClassifier, self).__init__()
        self.reccurrent = GRU(300, 300)
        self.layers = ModuleList(
            [
                Linear(300, 100, True),
                Linear(100, 1, True)
            ]
        )
        self.h0 = torch.zeros((1,300)).cuda()
    
    def forward(self, input):
        _, h =self.reccurrent(input, self.h0)
        return torch.sigmoid(self.layers[1](relu(self.layers[0](h[-1]))))


In [5]:
train, valid = random_split(dataset, [len(dataset)-len(dataset)//10 , len(dataset)//10])
dataloader = DataLoader(train, batch_size=1)
model = RecurrentClassifier()
model.cuda()

NameError: name 'dataset' is not defined

In [ ]:
optim = Adagrad(model.parameters())
loss = 0
losses = []
batch_size = 500
valid_x = [ for i in valid.indices]
with tqdm(dataloader) as tepoch:
    for i, (X, y) in enumerate(tepoch):
        pred = model(X[0])
        if y:
            loss -= torch.log(pred)
        else:
            loss += torch.log(pred)
        if (i+1)%batch_size==0:
            
            loss /= batch_size
            
            optim.zero_grad()
            loss.backward()
            optim.step()

            pred = torch.stack([ model(x) for x, y in valid])
            accuracy = (pred == dataset.Y[valid.indices]).sum().item()/pred.shape[0]
            tepoch.set_postfix({'loss':loss.item(),'accuracy':accuracy})
            tepoch.refresh()
            losses.append(loss)
            loss = 0

  0%|          | 174/36000 [00:31<1:47:26,  5.56it/s]


KeyboardInterrupt: 

In [ ]:
from matplotlib import pyplot as plt

x = range(1, len(losses)+1)
y = torch.stack(losses).detach().cpu().numpy()

In [ ]:
pred = torch.round(pred)
pred = torch.reshape(pred,(pred.shape[0],))

0.495

tensor([[-1.],
        [-1.],
        [-1.],
        ...,
        [-1.],
        [-1.],
        [-1.]], device='cuda:0', grad_fn=<RoundBackward0>)